In [85]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
import scipy
from scipy.special import expit
from generate_data import get_sampler, get_missing_data_sample
from missing_data import add_cumulative_estimators, mean_estimator_plotter, get_mean_estimators, eval_estimators
from numpy.random import default_rng
import seaborn as sns
#sns.set_theme()
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt

In [87]:
# Distribution parameters
sigma = 1
sample_size = 1000

In [105]:
random_seed = 1000
r = default_rng(random_seed)
sampler = get_sampler(dist="SeaVan1", sigma=sigma)
d = sampler(sample_size)
sns.jointplot(data=d, x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [106]:
d

,x,obs_prob,obs,y,obs_prob_pred
0,1,0.500000,True,1.791703,0.519540
1,2,0.017986,False,2.370951,0.027245
2,2,0.017986,False,3.255922,0.027245
3,0,0.982014,True,1.171458,0.976607
4,0,0.982014,True,-1.620135,0.976607
...,...,...,...,...,...
995,1,0.500000,True,1.068853,0.519540
996,0,0.982014,True,0.872937,0.976607
997,2,0.017986,False,1.335519,0.027245
998,2,0.017986,False,3.149841,0.027245


A sample from the data generating distribution is plotted above. (The x values in the scatter plot are jittered so it's easier to see the distribution.) Our objective is to estimate the average value of y for this dataset. Unfortunately, we only get a biased sample. When obs is true, we observe x and y, but when obs is false, we only observe x. Below is a plot of the observed data:

In [90]:
sns.jointplot(data=d[d["obs"]], x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Below we take a large sample to get a good estimate of the true population mean:

In [91]:
d_big = sampler(n=10000000)
seavan1_full_data_mean = np.mean(d_big['y'])
print(f"Mean of all y's from sample of size {d_big.shape[0]}: {seavan1_full_data_mean} ")

Mean of all y's from sample of size 10000000: 1.0006397361041992 


If we naively take the mean of the observed y's, we get a heavily biased estimate:

In [92]:
obs_mean = np.mean(d[d["obs"]]['y'])
print(f"Mean of all _observed_ y's from sample of size {d.shape[0]}: {obs_mean} ")

Mean of all _observed_ y's from sample of size 1000: 0.3223935079515721 


Now we'll generate some other estimators and see how they do.

In [107]:
sampler = get_sampler(dist="SeaVan1", sigma=sigma, rng = default_rng(1000))
d = sampler(sample_size)
d = add_cumulative_estimators(d)
fig, axs = plt.subplots(2,1, sharex='col')
mean_estimator_plotter(d=d, id_col="n", value_cols=["ipw_est"], 
                       true_mean=seavan1_full_data_mean, include_scatter=False, 
                       param_dict={"markers":False}, ax=axs[0])

sns.scatterplot(data=d[d["obs"]], x="n", y="y", hue = "obs_prob", size="weight", ax=axs[1], sizes=(3, 100), legend=False) #

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [108]:
d

,x,obs_prob,obs,y,obs_prob_pred,n,cum_obs_cnt,cum_obs_val,obs_mean,weight,cum_obs_weight,cum_obs_weighted_y,ipw_est,ipw_est_b,ipw_est_b_b,ipw_est_b3
0,0,0.982014,True,3.243483,0.978823,1,1,3.243483,3.243483,1.021635,1.021635,3.313655,3.313655,3.243483,3.174797,3.107565
1,1,0.500000,False,0.167638,0.514341,2,1,3.243483,3.243483,1.944235,1.021635,3.313655,1.656827,3.243483,6.349593,12.430261
2,2,0.017986,False,1.911987,0.023691,3,1,3.243483,3.243483,42.210552,1.021635,3.313655,1.104552,3.243483,9.524390,27.968087
3,1,0.500000,False,0.097804,0.514341,4,1,3.243483,3.243483,1.944235,1.021635,3.313655,0.828414,3.243483,12.699187,49.721044
4,2,0.017986,False,1.445201,0.023691,5,1,3.243483,3.243483,42.210552,1.021635,3.313655,0.662731,3.243483,15.873983,77.689132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,0.500000,True,2.186609,0.514341,996,498,177.941570,0.357312,1.944235,956.706424,1023.398684,1.027509,1.069710,1.113645,1.159384
996,1,0.500000,False,0.475430,0.514341,997,498,177.941570,0.357312,1.944235,956.706424,1023.398684,1.026478,1.069710,1.114763,1.161714
997,0,0.982014,True,-0.937456,0.978823,998,499,177.004115,0.354718,1.021635,957.728059,1022.440947,1.024490,1.067569,1.112460,1.159238
998,1,0.500000,True,1.812067,0.514341,999,500,178.816182,0.357632,1.944235,959.672294,1025.964032,1.026991,1.069077,1.112889,1.158495


In [109]:
fig, axs = plt.subplots(2,1, sharex='col')
mean_estimator_plotter(d=d, id_col="n", value_cols=["ipw_est", "ipw_est_b"], 
                       true_mean=seavan1_full_data_mean, include_scatter=False, 
                       param_dict={"markers":False}, ax=axs[0])

sns.scatterplot(data=d[d["obs"]], x="n", y="y", hue = "obs_prob", size="weight", 
                ax=axs[1], sizes=(3, 100), legend=False) #

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [112]:
sampler = get_sampler(dist="SeaVan1", sigma=sigma, rng = default_rng(5))
num_repeats = 5000  #use 5000 for better SEs
data_list = []
num_obs_list = []
for i in range(num_repeats):
    ## Randomly choose observations
    d = sampler(n=sample_size)
    est = get_mean_estimators(vals=d['y'],
                              covar=d[["x"]],
                              propensities=d["obs_prob_pred"],
                              obs=d['obs'])
    data_list.append(est)
    num_obs_list.append(d['obs'].sum())
results_seavan1 = pd.DataFrame(data_list)

In [113]:
results_seavan1

,obs_mean,ipw_mean,ipw_b_mean,weight,ipw_b_b_mean,ipw_b3_mean,num_obs,impute_missing,impute_missing_w,ipw_dr_1,ipw_dr_w_1,ipw_dr_opt,ipw_dr_w_opt,ipw_dr_b,ipw_dr_w_b
0,0.382735,0.963568,1.019375,945.253452,1.078415,1.140873,510,0.991868,1.090555,1.075735,1.090515,1.075735,1.090515,1.016843,1.030813
1,0.379395,0.711415,0.850950,836.024792,1.017853,1.217491,491,1.015611,1.056506,1.039222,1.056313,1.039222,1.056313,0.868815,0.883104
2,0.310570,0.784665,0.794219,987.970200,0.803890,0.813678,519,0.968681,0.804136,0.810697,0.805114,0.810697,0.805114,0.800945,0.795429
3,0.436423,0.959100,1.104992,867.970480,1.273075,1.466726,496,1.068824,1.330524,1.237644,1.329185,1.237644,1.329185,1.074239,1.153694
4,0.348406,0.667779,0.795934,838.988115,0.948683,1.130747,488,1.034246,0.989781,1.009083,0.990514,1.009083,0.990514,0.846608,0.831029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.369370,0.850506,0.916308,928.187463,0.987202,1.063580,505,0.995311,0.998259,0.997714,0.998281,0.997714,0.998281,0.926066,0.926592
4996,0.358421,0.898404,0.964907,931.078626,1.036333,1.113045,503,1.153003,1.048519,1.067374,1.048408,1.067374,1.048408,0.993809,0.976151
4997,0.457541,0.839753,0.882616,951.436465,0.927667,0.975017,510,1.026385,0.924646,0.937276,0.923834,0.937276,0.923834,0.891758,0.878969
4998,0.325026,0.451213,0.604415,746.529351,0.809633,1.084529,493,0.968133,0.919319,0.951092,0.919392,0.951092,0.919392,0.710018,0.686353


In [114]:
results = results_seavan1
x_var, y_var = ("weight", "ipw_mean")
r, p = scipy.stats.pearsonr(results[x_var], results[y_var])
print(f"r: {r}, p: {p}")
sns.jointplot(data=results, x=x_var, y=y_var, kind="reg", x_jitter=0, scatter_kws={"s": 1})
x_var, y_var = ("weight", "ipw_b_mean")
r, p = scipy.stats.pearsonr(results[x_var], results[y_var])
print(f"r: {r}, p: {p}")
sns.jointplot(data=results, x=x_var, y=y_var, kind="reg", x_jitter=0, scatter_kws={"s": 1})
x_var, y_var = ("weight", "ipw_b_b_mean")
r, p = scipy.stats.pearsonr(results[x_var], results[y_var])
print(f"r: {r}, p: {p}")
sns.jointplot(data=results, x=x_var, y=y_var, kind="reg", x_jitter=0, scatter_kws={"s": 1})

r: 0.7738762097340914, p: 0.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

r: 0.5767898921072427, p: 0.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

r: 0.17726255854974424, p: 1.4178025757107995e-36


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [116]:
results = results_seavan1
estimators_to_keep = ["ipw_mean", "ipw_b_mean"]
a = results[estimators_to_keep].apply(eval_estimators, axis=0, raw=True, result_type="expand", 
                                      true_val=seavan1_full_data_mean)
results_eval = pd.DataFrame(a.to_dict())
print(results_eval)
print(f"Mean number of observed y values: {np.mean(num_obs_list)} with SD: ")

              ipw_mean  ipw_b_mean
mean          0.828959    0.897344
SD            0.179947    0.151429
SE            0.002545    0.002142
bias         -0.171681   -0.103295
mean_abs_err  0.205642    0.146551
RMS error     0.248707    0.183305
Mean number of observed y values: 499.9532 with SD: 


To generate the table above, we repeated the following num_repeats times: 1) Get a sample of size sample_size from the distribution. 2) Evaluate estimators for the mean of y on this sample. The table then displays various summary statistics of the performance of these estimators across the num_repeats samples. Mean and SD are just the mean and SD of the estimate across the repeats. Our estimate of the expected value of the estimates are the mean and the SE is the uncertainty of that estimate. Bias is the difference between the true_mean and the mean of the estimators. The SE is also the uncertainty of this bias. (We've assumed that our estimate of the true mean has negligible error.) For each trial we compute the absolute and RMS difference between the estimate and the "true mean" (which itself was estimated, but from a large sample size)

The bias is relatively small for both estimators, but the bias of ipw_mean is much less than 1 SE from 0, while ipw_b_mean is more than 2 SEs from 0. As expected, ipw_b_mean has a small but significant bias. For both estimators, the SD is more than 10x the bias, which is reflected in the RMS error being essentially equal to the SD for each estimator. The ipw_b_mean has significantly smaller SD.

In [118]:
results = results_seavan1
estimators_to_keep = ["ipw_mean", "ipw_b_mean", "ipw_b_b_mean"]
a = results[estimators_to_keep].apply(eval_estimators, axis=0, raw=True, 
                                      result_type="expand", true_val=seavan1_full_data_mean)
results_eval = pd.DataFrame(a.to_dict())
print(results_eval)
print(f"Mean number of observed y values: {np.mean(num_obs_list)} with SD: ")

              ipw_mean  ipw_b_mean  ipw_b_b_mean
mean          0.828959    0.897344      0.976601
SD            0.179947    0.151429      0.137767
SE            0.002545    0.002142      0.001948
bias         -0.171681   -0.103295     -0.024039
mean_abs_err  0.205642    0.146551      0.109778
RMS error     0.248707    0.183305      0.139849
Mean number of observed y values: 499.9532 with SD: 


The twice self-normalized estimator has improved performance, presumably by removing the rest of the variation caused by correlation between the total weight and the estimate. More precisely, conditional on the total weight, the ipw and even the ipw_b estimators have a significant bias that is positively correlated with the weight. When we combine all these estimates together, the positive and negative biases inflate the overall variance of the estimator. I think the takeaway here is that there is no estimator that is "best" for all distributions. The ipw_b_b estimator happens to work quite well for this distribution, because even self-normalizing didn't review all of the correlation with the weight. But this isn't the case for other distributions.

We now consider a different kind of estimator: imputation estimators. The approach we consider here is to build a model for the expectation of Y given X, using the complete cases. Then impute the predictions for the incomplete cases. We then take the average of the Y's and imputed Y's, as though it were the full data set. For illustration, below we consider a linear fit of the data.

In [119]:
results = results_seavan1
estimators_to_keep = ["ipw_mean", "ipw_b_mean", "impute_missing", "impute_missing_w"]
a = results[estimators_to_keep].apply(eval_estimators, axis=0, raw=True, 
                                      result_type="expand", true_val=seavan1_full_data_mean)
results_eval = pd.DataFrame(a.to_dict())
print(results_eval)
print(f"Mean number of observed y values: {np.mean(num_obs_list)} with SD: ")

              ipw_mean  ipw_b_mean  impute_missing  impute_missing_w
mean          0.828959    0.897344        0.998938          0.997530
SD            0.179947    0.151429        0.077668          0.144599
SE            0.002545    0.002142        0.001098          0.002045
bias         -0.171681   -0.103295       -0.001702         -0.003110
mean_abs_err  0.205642    0.146551        0.061670          0.113082
RMS error     0.248707    0.183305        0.077686          0.144632
Mean number of observed y values: 499.9532 with SD: 


With impute_missing, we build a regression estimator for the missing values. The regression estimator has only 2 degrees of freedom and roughly sample_size/2 observations. This will lead to very low variance estimate of the regression model. However, more is reflected in SD than just that variance. In general, even with infinite data, the regression prediction may be wrong for certain x's because of model bias -- i.e. the true conditional distribution is not represented well in our class of models. These errors, while a consistent "bias" for individual x's, just look like noise when averaged across the x's. Thus this kind of error may also show up in the SD. If the errors for each x do not cancel out, on average, then we will end up with a bias. We will see this happening when we have data missing at random, with a model mismatch, and we don't propensity weight the training examples for the regession.

For the impute_missing_w estimator, we're using inverse propensity weighting in fitting the linear regression. We would expect this to potentially decrease the bias of the estimator at the expense of an increase in variance. However, this model has no bias because it perfectly matches the distribution. The bias doesn't seem much bigger than 0 (in terms of SE's), though the SD and RMSE is twice as large than the imputation estimator without IPW.

For this particular distribution, imputing with a linear model works tremendously well. Both the SD and the bias, as well as our two error measures, are the lowest on the board. However, this is largely due to the fact that a linear model is a perfect fit for the data distribution. We'll now repeat this experiment with the modified dataset, in which the conditional mean of y is not a linear function of x. Now Y has mean 1 whether x=1.0 or 2.0, but has mean 0 for x=0.0.

In [120]:
sigma = 1
random_seed = 1000
sample_size = 1000
r = default_rng(random_seed)
sampler = get_sampler(dist="SeaVan2", sigma=sigma)
d = sampler(sample_size)
sns.jointplot(data=d, x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})
sns.jointplot(data=d[d["obs"]], x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})
d_big = sampler(n=10000000)
seavan2_full_data_mean = np.mean(d_big['y'])
print(f"Mean of all y's from sample of size {d_big.shape[0]}: {seavan2_full_data_mean} ")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean of all y's from sample of size 10000000: 0.6673925215544576 


The first plot above is of the full dataset, while the second is from the observed portion of the dataset. An unweighted linear fit is given through each set of points. Note that the fit changes substantially when we leave out the missing data, as opposed to the first data distribution we examined. The hope is that by inverse propensity weighting the observed points in the linear fit, we can get a better approximation to the linear fit of the fully observed data.

In the first scatter plot, we see that the best a linear fit can do with this nonlinear relation. The mismatch of the model to the data gives rise to a bias that we will call model bias. In the second scatter plot, which shows the data we actually observe, we see that the linear fit significantly overestimates the mean of y for x=2. Here the model bias is exacerbated by a sample bias. Here are the results for our various estimators on this dataset.

In [123]:
num_repeats = 1000  #use 5000 for better SEs
random_seed = 1000
rng = default_rng(random_seed)
sampler = get_sampler(dist="SeaVan2", sigma=sigma, rng = rng)
data_list = []
num_obs_list = []
for i in range(num_repeats):
    ## Randomly choose observations
    d = sampler(n=sample_size)
    est = get_mean_estimators(vals=d['y'],
                              covar=d[["x"]],
                              propensities=d["obs_prob_pred"],
                              obs=d['obs'])
    data_list.append(est)
    num_obs_list.append(d['obs'].sum())
results_seavan2 = pd.DataFrame(data_list)

In [124]:
results = results_seavan2
estimators_to_keep = ["ipw_mean", "ipw_b_mean", "impute_missing", 
                      "impute_missing_w"]
a = results[estimators_to_keep].apply(eval_estimators, axis=0, 
                                      raw=True, result_type="expand", true_val=seavan2_full_data_mean)
results_eval = pd.DataFrame(a.to_dict())
print(results_eval)
print(f"Mean number of observed y values: {np.mean(num_obs_list)} with SD: ")

              ipw_mean  ipw_b_mean  impute_missing  impute_missing_w
mean          0.586963    0.636445        0.940288          0.692951
SD            0.125457    0.119475        0.079358          0.141605
SE            0.003967    0.003778        0.002510          0.004478
bias         -0.080429   -0.030947        0.272896          0.025559
mean_abs_err  0.121969    0.097684        0.272896          0.111968
RMS error     0.149025    0.123418        0.284200          0.143893
Mean number of observed y values: 500.023 with SD: 


On this dataset, we see things according with our expectations: the bias of ipw_mean is indistinguishable from 0 (within 1 SE), but has the highest SD. ipw_b_mean has controlled the variance of ipw_mean to some extent, but at the cost of an increase in bias, which is significantly different from 0. Still, the SD is more than an order of magnitude larger than the bias, so the tradeoff is a good one. This is reflected in the improved mean_abs_error and RMS error.

Meanwhile impute_missing has a very large bias. Because of the sample bias, in this case the very small sample probability when x=2, the linear fit is quite a bit off from the complete data case. On the other hand, the SD is low. This is because the model requires relatively small sample size for a stable fit.

With impute_missing_w, where we inverse propensity weight the points, the sample bias is largely removed. However, the large weights on the relatively few number of points make the effective sample size much smaller. This inflates the SD. This tradeoff of getting a more variance for substantially less bias was a good one for impute_missing_w compared to impute_missing, as is reflected in their relative performance.

In [125]:
results = results_seavan2
estimators_to_keep = ["ipw_mean", "ipw_b_mean", "impute_missing", 
                      "impute_missing_w", "ipw_dr_1", "ipw_dr_w_1", "ipw_dr_opt", "ipw_dr_w_opt",
                      "ipw_dr_b", "ipw_dr_w_b",]
a = results[estimators_to_keep].apply(eval_estimators, axis=0, 
                                      raw=True, result_type="expand", true_val=seavan2_full_data_mean)
results_eval = pd.DataFrame(a.to_dict())
print(results_eval)
print(f"Mean number of observed y values: {np.mean(num_obs_list)} with SD: ")

              ipw_mean  ipw_b_mean  impute_missing  impute_missing_w  \
mean          0.586963    0.636445        0.940288          0.692951   
SD            0.125457    0.119475        0.079358          0.141605   
SE            0.003967    0.003778        0.002510          0.004478   
bias         -0.080429   -0.030947        0.272896          0.025559   
mean_abs_err  0.121969    0.097684        0.272896          0.111968   
RMS error     0.149025    0.123418        0.284200          0.143893   

              ipw_dr_1  ipw_dr_w_1  ipw_dr_opt  ipw_dr_w_opt  ipw_dr_b  \
mean          0.743039    0.692893    0.743039      0.692893  0.679070   
SD            0.132958    0.141576    0.132958      0.141576  0.107079   
SE            0.004205    0.004477    0.004205      0.004477  0.003386   
bias          0.075647    0.025501    0.075647      0.025501  0.011678   
mean_abs_err  0.122442    0.111963    0.122442      0.111963  0.084487   
RMS error     0.152972    0.143854    0.152972     

Let's consider the specific case in the paper, Examble 1(b), with sigma = 1 and n=100,000. In the paper, they also make a little logistic regression model to estimate the propensity score (i.e. the probability of getting na observation, conditioned on x). Here we assume propensities are known, which is why our variances are smaller for the IPW estimators.

In [126]:
num_repeats = 1000  #use 5000 for better SEs
random_seed = 1000
rng = default_rng(random_seed)
sampler = get_sampler(dist="SeaVanMod2", sigma=sigma, rng = rng)
data_list = []
num_obs_list = []
for i in range(num_repeats):
    ## Randomly choose observations
    d = sampler(n=sample_size)
    est = get_mean_estimators(vals=d['y'],
                              covar=d[["x"]],
                              propensities=d["obs_prob_pred"],
                              obs=d['obs'])
    data_list.append(est)
    num_obs_list.append(d['obs'].sum())
results_seavanMod2 = pd.DataFrame(data_list)

In [127]:
d_big = sampler(n=10000000)
seavanmod2_full_data_mean = np.mean(d_big['y'])
print(f"Mean of all y's from sample of size {d_big.shape[0]}: {seavanmod2_full_data_mean} ")

Mean of all y's from sample of size 10000000: 1.6676854683793805 


In [128]:
sns.jointplot(data=d, x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})
sns.jointplot(data=d[d["obs"]], x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [129]:
results = results_seavanMod2
estimators_to_keep = ["ipw_mean", "ipw_b_mean", "impute_missing", 
                      "impute_missing_w"]
a = results[estimators_to_keep].apply(eval_estimators, axis=0, 
                                      raw=True, result_type="expand", true_val=seavanmod2_full_data_mean)
results_eval = pd.DataFrame(a.to_dict())
print(results_eval)
print(f"Mean number of observed y values: {np.mean(num_obs_list)} with SD: ")

              ipw_mean  ipw_b_mean  impute_missing  impute_missing_w
mean          1.340515    1.443464        1.125976          1.626516
SD            0.295947    0.235740        0.092203          0.157587
SE            0.009359    0.007455        0.002916          0.004983
bias         -0.327170   -0.224222       -0.541709         -0.041169
mean_abs_err  0.366312    0.262279        0.541709          0.127758
RMS error     0.441163    0.325344        0.549500          0.162876
Mean number of observed y values: 500.023 with SD: 


In [131]:
num_repeats = 1000  #use 5000 for better SEs
random_seed = 1000
sigma = 1
obs_prob = 0.1
rng = default_rng(random_seed)
sampler = get_sampler(dist="MCAR_normal_sqr", sigma=sigma, obs_prob=obs_prob, rng = rng)
data_list = []
num_obs_list = []
for i in range(num_repeats):
    ## Randomly choose observations
    d = sampler(n=sample_size)
    est = get_mean_estimators(vals=d['y'],
                              covar=d[["x"]],
                              propensities=d["obs_prob_pred"],
                              obs=d['obs'])
    data_list.append(est)
    num_obs_list.append(d['obs'].sum())
results_MCAR_normal_sqr = pd.DataFrame(data_list)
d_big = sampler(n=10000000)
MCAR_normal_sqr_full_data_mean = np.mean(d_big['y'])
print(f"Mean of all y's from sample of size {d_big.shape[0]}: {MCAR_normal_sqr_full_data_mean} ")

Mean of all y's from sample of size 10000000: 3.002234668379598 


In [132]:
results = results_MCAR_normal_sqr
estimators_to_keep = ["ipw_mean", "ipw_b_mean", "impute_missing"]
a = results[estimators_to_keep].apply(eval_estimators, axis=0, 
                                      raw=True, result_type="expand", true_val=MCAR_normal_sqr_full_data_mean)
results_eval = pd.DataFrame(a.to_dict())
print(results_eval)
print(f"Mean number of observed y values: {np.mean(num_obs_list)} with SD: ")

              ipw_mean  ipw_b_mean  impute_missing
mean          2.997494    2.997883        2.993700
SD            0.197088    0.196488        0.196990
SE            0.006232    0.006213        0.006229
bias         -0.004741   -0.004352       -0.008535
mean_abs_err  0.157111    0.156575        0.157176
RMS error     0.197145    0.196536        0.197175
Mean number of observed y values: 99.758 with SD: 


In [133]:
sns.jointplot(data=d, x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …